In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.api import SimpleExpSmoothing

data = pd.read_csv('d:\Projects\dataset\Biomass_History.csv', index_col=0)
dist_matrix = pd.read_csv("D:\Projects\dataset\Distance_Matrix.csv", index_col=0)
testdata = data.copy()

In [ ]:
years = list(data.columns[2:])
def simpleexp(row):
    model = SimpleExpSmoothing(row[years])
    model_fit = model.fit()
    row['2018'] = model_fit.forecast(2)[0]
    row['2019'] = model_fit.forecast(2)[1]
    return row

testdata = testdata.apply(simpleexp, axis=1)

In [ ]:
#testdata.to_excel(r'D:\Projects\dataset\forecasted.xlsx')

In [ ]:
import gurobipy as gp 
from gurobipy import GRB
from itertools import product

num_biomass = num_depot = testdata['2018'].size
cartesian_prod = list(product(range(num_biomass), range(num_depot)))

# model
model = gp.Model('facility_location')

In [ ]:
# variables 
y_select = model.addVars(num_depot, vtype = GRB.BINARY, name = 'SELECT')
biomass_assign = model.addVars(cartesian_prod, vtype = GRB.CONTINUOUS, name = 'ASSIGN')

# objective
model.setObjective((gp.quicksum(dist_matrix.values[(i,j)]*biomass_assign[(i,j)] for i, j in cartesian_prod)) + gp.quicksum((25000-gp.quicksum(biomass_assign[(i,j)] for i in range(num_biomass)))*y_select[j] for j in range(num_depot)), GRB.MINIMIZE)

# constraints
model.addConstrs((gp.quicksum(biomass_assign[(i,j)] for i in range(num_biomass)) <= 25000*y_select[j] for j in range(num_depot)), name = 'capacity')
model.addConstr((gp.quicksum(y_select[j] for j in range(num_depot)) <= 25), 'limitdepot')
model.addConstrs((gp.quicksum(biomass_assign[(i,j)] for j in range(num_depot)) <= testdata['2018'][i] for i in range(num_biomass)), name = 'lessthanforecast')
model.addConstrs((gp.quicksum(biomass_assign[(i,j)] for j in range(num_depot)) >= 0.8* testdata['2018'][i] for i in range(num_biomass)), name = 'util80')

model.optimize()